In [5]:
from tagger.models.received_item import ReceivedItem, ReceivedItemStatus
from tagger.core.mongo.models.order import OrderCodeMap, Order, OrderStatus
from tagger.viewsets.order.change_status import ChangeStatusSerializer, _change_status
from django.contrib.auth.models import User
from tagger.models.inventory import Inventory, InventoryStatus

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import pandas as pd

def split_codes(raw_codes):
    codes = []
    cursor = 0
    while cursor < len(raw_codes):
        codes.append(raw_codes[cursor: cursor + 5])
        cursor = cursor + 5
    return codes

In [2]:
with open("../Downloads/확장주문검색_20211108_161813.xlsx", "rb") as f:
    df = pd.read_excel(f, dtype=str)
    df.dropna(subset=["주문번호"], inplace=True)
    for _, row in df.iterrows():
        print(row["주문번호"], row["판매처 상품명"], row["수령자이름"])

ABC01 Wool and cashmere cardigan(Camel beige) 이남이
CHLVA 래시 프린지 캐시미어 울 머플러 정희리ㅣ
UCJ4A 래시 프린지 캐시미어 울 머플러 손소명
3WF6L [재입고] 래시 프린지 캐시미어 울 머플러 조하은
M36BC 체크 트위드 자켓 최하림
UEA8Q 후드 배색 니트 드레스 남도연
D7G6K 컬러 블록 스웻셔츠(스모크 블루) 최부영
DTX3E 언밸런스 니트 스커트 이나래
XGJN3 백 플랩 니트 드레스(COCOA) 김혜경
K493A 알파카 니트 드레스(WHITENED PINK) 정혜영
DWNMC 캐시미어 하이넥 니트 김지현(박숙연)
PPCCU 라벨링 롱 코트 유연주
N7PMH 언밸런스 울 스커트 원혜림
WFUNM 핸드메이드 캐시미어 코트 (그레이) 나춘영
963Y8 숄 블록 니트 탑 고서현


In [3]:
for _, row in df.iterrows():
    codes = split_codes(row["주문번호"])
    tracking_no = f"{row['택배사']} {row['송장번호']}"
    tracking_url = f"https://search.naver.com/search.naver?query={tracking_no}"
    for code in codes:
        code_map = OrderCodeMap.objects(code=f"ORD-{code}").first()
        orderid = str(code_map.orderid)
        order = Order.objects.get(id=orderid)
        s = ChangeStatusSerializer(data={
            "status": OrderStatus.DELIVERY_STARTED,
            "delivery_tracking_number": tracking_no,
            "delivery_tracking_url": tracking_url,
        }, instance=order)
        s.is_valid(raise_exception=True)
        _change_status(s, order, User.objects.get(username="backoffice_server"), id=orderid)
        print(f"https://office.alloff.co/orders/{orderid}", order.get_payment().buyername)

https://office.alloff.co/orders/616bfc4c17fc2e5a1f9e8d24 이남이
https://office.alloff.co/orders/617ea9be17fc2e5a1f9eb1c9 정희리ㅣ
https://office.alloff.co/orders/617f434f17fc2e5a1f9eb207 손소명
https://office.alloff.co/orders/617f7f0817fc2e5a1f9eb240 조하은
https://office.alloff.co/orders/6180623717fc2e5a1f9eb433 최하림
https://office.alloff.co/orders/6180fe1b26685ff051754c57 남도연
https://office.alloff.co/orders/618106f426685ff051754c79 최부영
https://office.alloff.co/orders/618116fc26685ff051754ca1 이나래
https://office.alloff.co/orders/618263b826685ff051754f24 김혜경 
https://office.alloff.co/orders/61826e1926685ff051754f4d 정혜영
https://office.alloff.co/orders/6183a51926685ff05175516b 김지현<박숙연>
https://office.alloff.co/orders/61840ba426685ff05175532a 유연주
https://office.alloff.co/orders/6185270426685ff051755530 원혜림
https://office.alloff.co/orders/6185735326685ff0517555d8 나춘영
https://office.alloff.co/orders/6188765726685ff051755a57 고서현


In [6]:
for _, row in df.iterrows():
    codes = split_codes(row["주문번호"])
    for code in codes:
        order = Order.get_by_code(code)
        if order is not None:
            inventories = Inventory.objects.filter(status=InventoryStatus.SHIPPING_PENDING, out_order_id=str(order.id)).all()
            for i in inventories:
                print(f"""{order.code} ({order.id}) - {i.product_name} has been shipped!""")
                i.status = InventoryStatus.SHIPPED
                i.save()

ORD-ABC01 (616bfc4c17fc2e5a1f9e8d24) - Wool and cashmere cardigan(Camel beige) has been shipped!
ORD-CHLVA (617ea9be17fc2e5a1f9eb1c9) - 래시 프린지 캐시미어 울 머플러 has been shipped!
ORD-UCJ4A (617f434f17fc2e5a1f9eb207) - 래시 프린지 캐시미어 울 머플러 has been shipped!
ORD-3WF6L (617f7f0817fc2e5a1f9eb240) - [재입고] 래시 프린지 캐시미어 울 머플러 has been shipped!
ORD-M36BC (6180623717fc2e5a1f9eb433) - 체크 트위드 자켓  has been shipped!
ORD-UEA8Q (6180fe1b26685ff051754c57) - 후드 배색 니트 드레스  has been shipped!
ORD-D7G6K (618106f426685ff051754c79) - 컬러 블록 스웻셔츠(스모크 블루) has been shipped!
ORD-DTX3E (618116fc26685ff051754ca1) - 언밸런스 니트 스커트  has been shipped!
ORD-XGJN3 (618263b826685ff051754f24) - 백 플랩 니트 드레스(COCOA) has been shipped!
ORD-K493A (61826e1926685ff051754f4d) - 알파카 니트 드레스(WHITENED PINK) has been shipped!
ORD-DWNMC (6183a51926685ff05175516b) - 캐시미어 하이넥 니트  has been shipped!
ORD-PPCCU (61840ba426685ff05175532a) - 라벨링 롱 코트  has been shipped!
ORD-N7PMH (6185270426685ff051755530) - 언밸런스 울 스커트  has been shipped!
ORD-WFUNM (61857353266

True